*Luis Antonio Ortega Andrés   
Antonio Coín Castro*

In [1]:
import pandas as pd
import numpy as np
import time
from itertools import chain

# Gestión de Datos - Práctica 2

Disponemos de dos conjuntos de datos sobre valoraciones de películas que se actualizan diariamente, sobre los que queremos realizar de forma eficiente una consulta. La consulta que se pretende implementar a partir de los datos es una que retorne la lista de géneros, ordenados por el promedio de puntuación que han obtenido
en la última semana.

Se deberá realizar un trabajo de procesado de datos que al menos debe contener las
fases de carga, tratamiento de valores perdidos, normalización de los datos cuando sea
necesario y limpieza de outliers. Finalmente, los datos se deben almacenar de forma
estructurada que facilite la consulta propuesta.



## Procesamiento de los datos disponibles

### Carga de datos

El primer paso a seguir es cargar los datos disponibles en los ficheros `movies.csv` y `ratings.csv` en sendos dataframes para su exploración y procesamiento. En lo sucesivo nos referiremos al dataframe de películas como `dfm` y al de valoraciones como `dfr`.

In [2]:
dfm = pd.read_csv("movies.csv")
dfr = pd.read_csv("ratings.csv")

Echamos un vistazo a la configuración de los datos:

- Para las películas, vemos que existen tres campos: el identificador de película (`movieId`), el título de la misma (`title`) y una lista de géneros asociada a la misma, separada por "|" (`genres`). Hay un total de 62423 entradas.

- En cuanto a las valoraciones, existen cuatro campos relevantes: el identificador del usuario que realizó la valoración (`userId`), el identificador de película para cruzar esta tabla con la anterior (`movieId`), la valoración realizada sobre 5 estrellas e incrementos de media estrella (`rating`), y la fecha en la que se hizo, medida en segundos desde el 1 de enero de 1970 UTC (`timestamp`). Tenemos un total de 25000095 entradas, y destacamos que todas las columnas tienen valores numéricos (comprobado con la función `.info()`).

In [3]:
print("[dfm] Tamaño:", dfm.shape)
dfm.head()

[dfm] Tamaño: (62423, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
print("[dfr] Tamaño:", dfr.shape)
dfr.head()

[dfr] Tamaño: (25000095, 4)


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


### Datos duplicados

Antes de nada, debemos asegurarnos en la medida de lo posible que el conjunto de datos es consistente, y principalmente de que no hay entradas repetidas que sean contradictorias. Vamos a asumir que el sistema de recogida de datos no permite errores del tipo "un mismo usuario valora la misma película en dos ocasiones distintas", y en general vamos a ignorar errores de esta naturaleza que no afecten a nuestra consulta final. 

Lo único que hacemos a este respecto es eliminar las filas completamente repetidas.

In [5]:
rows_dfm = len(dfm)
rows_dfr = len(dfr)
dfm = dfm.drop_duplicates()
dfr = dfr.drop_duplicates()
print("[dfm] Filas duplicadas:", rows_dfm - len(dfm))
print("[dfr] Filas duplicadas:", rows_dfr - len(dfr))

[dfm] Filas duplicadas: 0
[dfr] Filas duplicadas: 8


### Valores perdidos

Para analizar los valores perdidos iremos campo a campo en cada dataframe.

**dfm**

Para las películas, si el campo `movieId` no está presente o no es numérico, descartamos toda la fila, ya que si no está presente no tenemos forma de saber cuál es el identificador de la película.

In [6]:
rows_dfm = len(dfm)
dfm = dfm[dfm.movieId.apply(lambda x: np.isreal(x))]
dfm = dfm.dropna(subset=['movieId'])
print("[dfm] Filas con movieId inválido:", rows_dfm - len(dfm))

[dfm] Filas con movieId inválido: 0


No nos importa el valor del campo `title` siempre que el `movieId` sea válido. En cuanto a `genres`, sabemos que deben estar separados por "|" cuando haya más de uno, y supondremos que solo pueden contener letras mayúsculas y minúsculas, guiones y espacios en blanco. En primer lugar miramos cuáles son los valores únicos en nuestro dataset para este campo.

In [7]:
a = [g.split("|") for g in list(dfm['genres'])]
np.unique(list(chain.from_iterable(a)))

array(['(no genres listed)', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Espionage Action', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Psychological Thriller', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'], dtype='<U22')

Vemos que la forma de indicar que no hay información disponible sobre el género es la cadena '(no genres listed)', por lo que si en alguna fila está presente, descartamos la fila completa (ya que no es útil para nuestra consulta final). No hay más valores discordantes, pero por si acaso, hacemos lo mismo buscando valores NaN en ese campo.

In [8]:
rows_dfm = len(dfm)
dfm = dfm[dfm['genres'] != '(no genres listed)']
dfm = dfm.dropna(subset=['genres'])
print("[dfm] Filas con genres no presente:", rows_dfm - len(dfm))

[dfm] Filas con genres no presente: 5062


**dfr**

El campo `userId` no nos importa que sea inválido para nuestra consulta final. Para el campo `movieId` hacemos lo mismo que antes, por el mismo motivo.

In [9]:
rows_dfr = len(dfr)
dfr = dfr.dropna(subset=['movieId'])
dfr = dfr[dfr.movieId.apply(lambda x: np.isreal(x))]
print("[dfr] Filas con movieId inválido:", rows_dfr - len(dfr))

[dfr] Filas con movieId inválido: 0


En cuanto a `rating`, como es la variable clave sobre la que vamos a hacer la consulta, debemos asegurarnos de que está presente y que tiene un valor válido (es decir, un número en $[0,5]$ que sea múltiplo de $0.5$). En otro caso descartamos la fila completa, ya que no tendría sentido imputar ningún valor, pues estaríamos modificando significativamente el promedio sin tener una razón estadística de peso que lo sugiriese.

Esto es así ya que la única información relevante de la tabla `dfr` es justamente la valoración, y sin ella toda la entrada se vuelve irrelevante.

In [10]:
rows_dfr = len(dfr)
dfr = dfr.dropna(subset=['rating'])
dfr = dfr[dfr.rating.apply(lambda x: np.isreal(x) and x >= 0 
                           and x <= 5 and int(2*x) == 2*x)]
print("[dfr] Filas con rating inválido:", rows_dfr - len(dfr))

[dfr] Filas con rating inválido: 505332


Por último, para el campo `timestamp` seguimos la misma filosofía: comprobamos que esté presente y que sea válido, pues de otro modo no tendremos forma de saber cuándo se produjo la valoración, que es algo que nos interesa en nuestra consulta final. En este caso, un valor válido será un número entre $0$ y el *timestamp* actual.

In [11]:
rows_dfr = len(dfr)
dfr = dfr.dropna(subset=['timestamp'])
dfr = dfr[dfr.timestamp.apply(lambda x: np.isreal(x) and x >= 0 
                           and x <= time.time())]
print("[dfr] Filas con timestamp inválido:", rows_dfr - len(dfr))

[dfr] Filas con timestamp inválido: 0


### Normalización de datos

En este caso no es necesario normalizar los datos. El único campo numérico que podría tener interés es el de `rating`, pero tras la limpieza realizada nos hemos asegurado que todos los valores se encuentran en la escala predefinida en $[0,5]$. Como en nuestra consulta final no vamos a comparar variables distintas, no es necesario normalizar.

### Limpieza de *outliers*

El único valor para el que tendría sentido buscar *outliers* es el de `rating`, y quizás el de `timestamp`.  Sin embargo, valores fuera del rango permitido para estas dos variables no serían *outliers*, sino directamente valores inválidos (que ya hemos limpiado previamente). Además, si hubiera valores dentro del rango permitido pero alejados de la mayoría, seguirían siendo igual de válidos y no podrían considerarse *outliers*, al menos en lo que respecta a nuestra consulta objetivo (no tendría sentido tratar distinto una puntuación de 0 estrellas, incluso aunque la mayoría de puntuaciones fueran de 5 estrellas, siempre y cuando nos hayamos asegurado de que son datos válidos).

### Estadísticas

Mostramos un pequeño resumen estadístico de los valores modificados de una forma u otra en el análisis y limpieza del dataset.

**dfm**

| Duplicados | Valores perdidos o inválidos | Outliers | 
|:----------:|:----------------------------:|:--------:|
| 0 | 5062 | 0 |

| Transformación (en orden) | Nº registros antes | Nº registros después |
|:--------------:|:------------------:|:------------------------:|
| Duplicados | 62423 | 62423 |
| Valores perdidos o inválidos | 62423 | 57361 |
| Outliers | 57361 | 57361 |

**dfr**

| Duplicados | Valores perdidos o inválidos | Outliers | 
|:----------:|:----------------------------:|:--------:|
| 8 | 505332 | 0 |

| Transformación (en orden) | Nº registros antes | Nº registros después |
|:--------------:|:------------------:|:------------------------:|
| Duplicados | 25000095 | 25000087 |
| Valores perdidos o inválidos | 25000087 | 24494755 |
| Outliers | 24494755 | 24494755 |

Concluimos que todos los valores eliminados del conjunto sea por la razón que sea constituyen un pequeño porcentaje del total en cada caso, por lo que en principio no tenemos por qué preocuparnos por haber perdido muchos datos.

## Procesamiento de datos futuros

Suponemos que estos datos se actualizan todos los días a las 3 AM hora local. Nuestro objetivo es realizar este mismo preprocesamiento y limpieza con los nuevos datos, y después convertirlos a un formato adecuado para poder realizar la consulta final. No haremos ninguna suposición sobre actualizaciones incrementales de los datos, es decir, supondremos que es posible que de un día al siguiente se modifiquen datos antiguos, en lugar de solo añadir datos nuevos. Esto significa también que no reutilizaremos información de un día para el siguiente.

El único campo que merece un poco más de atención es la lista de géneros en la tabla de películas. **Supondremos que los datos futuros seguirán la misma expresión regular que los actuales**, es decir, los géneros se separarán por "|" y estarán formados únicamente por letras mayúsculas y minúsculas, guiones y espacios. Cualquier otra cadena se considerará un valor inválido. En el script realizado se implementa la comprobación de esta expresión regular.

El tratamiento automatizado de los datos se implementa en el script `limpieza.py`, y la conversión al formato deseado en `conv.py`.

## Integración con airflow